In [1]:
import pandas as pd

In [3]:
evidence_df = pd.read_csv('/Users/darylwilding-mcbride/Downloads/190719_Hela_Ecoli-combined/txt_noMatching/evidence.txt', sep='\\t')

# remove decoys, which are indicated by a '+' in the Reverse column
evidence_df = evidence_df[pd.isna(evidence_df.Reverse)]

/Users/darylwilding-mcbride/anaconda3/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.


In [4]:
evidence_df.columns

Index(['Sequence', 'Length', 'Modifications', 'Modified sequence',
       'Oxidation (M) Probabilities', 'Oxidation (M) Score Diffs',
       'Acetyl (Protein N-term)', 'Oxidation (M)', 'Missed cleavages',
       'Proteins', 'Leading proteins', 'Leading razor protein', 'Gene names',
       'Protein names', 'Type', 'Raw file', 'Experiment', 'MS/MS m/z',
       'Charge', 'm/z', 'Mass', 'Resolution',
       'Uncalibrated - Calibrated m/z [ppm]',
       'Uncalibrated - Calibrated m/z [Da]', 'Mass error [ppm]',
       'Mass error [Da]', 'Uncalibrated mass error [ppm]',
       'Uncalibrated mass error [Da]', 'Max intensity m/z 0', 'Retention time',
       'Retention length', 'Calibrated retention time',
       'Calibrated retention time start', 'Calibrated retention time finish',
       'Retention time calibration', 'Ion mobility index',
       'Ion mobility length', '1/K0', '1/K0 length', 'Calibrated 1/K0', 'CCS',
       'CCS length', 'Match time difference', 'Match m/z difference',
       '

In [5]:
len(evidence_df.Sequence.unique())

27901

In [6]:
len(evidence_df['Protein names'].unique())

5575

In [8]:
evidence_df.Sequence.unique()

array(['AAAAAAAAAPAAAATAPTTAATTAATAAQ', 'AAAAAAAAVPSAGPAGPAPTSAAGR',
       'AAAAAAGAASGLPGPVAQGLK', ..., 'YYTSASGDEMVSLK', 'YYVTIIDAPGHR',
       'YYYAVVDCDSPETASK'], dtype=object)